# Smile Detection

In [2]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [3]:
import cv2 as cv

face_detector = cv.CascadeClassifier(os.path.join(cv.data.haarcascades, "haarcascade_frontalface_default.xml"))

In [5]:
import keras

model = keras.models.load_model("smile.keras")

In [10]:
import numpy as np

capture = cv.VideoCapture(0)

frames = []

try:
    while True:
      _, frame = capture.read()
    
      frame = cv.flip(frame, 1)
      frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

      face = face_detector.detectMultiScale(frame_gray, minNeighbors=10, minSize=(50, 50), scaleFactor=1.1)

      for x, y, w, h in face:
          face = cv.resize(frame_gray[y:y + h, x:x + w], (64, 64), interpolation=cv.INTER_CUBIC)
          face_array = np.array([face]) / 255

          smiling = model.predict(face_array)[0][1]

          cv.putText(
            frame,
            f"{round(smiling * 100)}%",
            (int(x), int(y - 5)),
            cv.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 255))

          if (smiling > 0.1):
              cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
          else:
              cv.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)

      cv.imshow("frame", frame)

      frames.append(frame)

      if cv.pollKey() == 27:
          break
    
      if cv.getWindowProperty("frame", cv.WND_PROP_VISIBLE) < 1:
          break

    cv.destroyAllWindows()
finally:
    capture.release()

for frame_index, frame in enumerate(frames):
    cv.imwrite(f"{frame_index:04}.png", frame)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step


QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is not the object's thread (0x2d944dd0).
Cannot move to target thread (0x2d91ef70)

QObject::moveToThread: Current thread (0x2d91ef70) is n

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━